In [86]:
import pandas as pd
from yahoofinancials import YahooFinancials

selected_date = "2024-05-13"
# selected_date = "2020-09-01"
date = pd.to_datetime(selected_date)
start_date = str((date - pd.Timedelta(days=13)).date())
end_date = str(date.date())
yahoo_financials = YahooFinancials("BTC-USD")
df = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
prices = df["BTC-USD"]["prices"]
prices

[{'date': 1714435200,
  'high': 64703.33203125,
  'low': 59120.06640625,
  'open': 63839.41796875,
  'close': 60636.85546875,
  'volume': 37840840057,
  'adjclose': 60636.85546875,
  'formatted_date': '2024-04-30'},
 {'date': 1714521600,
  'high': 60780.5,
  'low': 56555.29296875,
  'open': 60609.49609375,
  'close': 58254.01171875,
  'volume': 48439780271,
  'adjclose': 58254.01171875,
  'formatted_date': '2024-05-01'},
 {'date': 1714608000,
  'high': 59602.296875,
  'low': 56937.203125,
  'open': 58253.703125,
  'close': 59123.43359375,
  'volume': 32711813559,
  'adjclose': 59123.43359375,
  'formatted_date': '2024-05-02'},
 {'date': 1714694400,
  'high': 63320.50390625,
  'low': 58848.3125,
  'open': 59122.30078125,
  'close': 62889.8359375,
  'volume': 33172023048,
  'adjclose': 62889.8359375,
  'formatted_date': '2024-05-03'},
 {'date': 1714780800,
  'high': 64494.95703125,
  'low': 62599.3515625,
  'open': 62891.03125,
  'close': 63891.47265625,
  'volume': 20620477992,
  'adjcl

In [87]:
# Preprocess for prediction
df = pd.DataFrame(prices)
df["date"] = pd.to_datetime(df["formatted_date"])
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df.dropna(inplace=True)
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df.rename(columns={"volume": "Volume"}, inplace=True)
for i in range(1, 8):
    df[f"lag_close_{i}"] = df["close"].shift(i)
df.dropna(inplace=True)
df

,date,high,low,open,close,Volume,adjclose,formatted_date,lag_close_1,lag_close_2,lag_close_3,lag_close_4,lag_close_5,lag_close_6,lag_close_7
7,2024-05-07,64390.457031,62285.980469,63162.761719,62334.816406,2.593073e+10,62334.816406,2024-05-07,63161.949219,64031.132812,63891.472656,62889.835938,59123.433594,58254.011719,60636.855469
8,2024-05-08,62986.085938,60877.128906,62332.640625,61187.941406,2.608817e+10,61187.941406,2024-05-08,62334.816406,63161.949219,64031.132812,63891.472656,62889.835938,59123.433594,58254.011719
9,2024-05-09,63404.914062,60648.074219,61191.199219,63049.960938,2.545334e+10,63049.960938,2024-05-09,61187.941406,62334.816406,63161.949219,64031.132812,63891.472656,62889.835938,59123.433594
10,2024-05-10,63446.742188,60208.781250,63055.191406,60792.777344,2.780495e+10,60792.777344,2024-05-10,63049.960938,61187.941406,62334.816406,63161.949219,64031.132812,63891.472656,62889.835938
11,2024-05-11,61451.152344,60492.625000,60793.355469,60793.710938,1.384227e+10,60793.710938,2024-05-11,60792.777344,63049.960938,61187.941406,62334.816406,63161.949219,64031.132812,63891.472656
12,2024-05-12,61818.156250,60632.601562,60793.503906,61448.394531,1.380046e+10,61448.394531,2024-05-12,60793.710938,60792.777344,63049.960938,61187.941406,62334.816406,63161.949219,64031.132812
13,2024-05-13,61818.156250,60632.601562,60793.503906,61448.394531,1.380046e+10,61448.394531,2024-05-12,61448.394531,60793.710938,60792.777344,63049.960938,61187.941406,62334.816406,63161.949219


In [88]:
from joblib import load

# Load trained model from a file
try:
    file_name = "trained_models/bitsmart_model_1715639499"
    models = load(file_name)
    model_open = models["open"]
    model_high = models["high"]
    model_low = models["low"]
    model_close = models["close"]
    model_avg = models["average"]
except Exception as e:
    print(e)

In [89]:
def predict_next_7_days(selected_date, data):
    selected_date = pd.to_datetime(selected_date)
    prediction_results = []
    for i in range(7):
        base_date = selected_date - pd.Timedelta(days=i)
        record = data[data["date"] == base_date]
        feature_names = [
            "lag_close_1",
            "lag_close_2",
            "lag_close_3",
            "lag_close_4",
            "lag_close_5",
            "lag_close_6",
            "lag_close_7",
            "Volume",
        ]
        feature_vector = record[feature_names].values.reshape(1, -1)
        feature_vector_df = pd.DataFrame(feature_vector, columns=feature_names)
        predicted_open = model_open.predict(feature_vector_df)[0]
        predicted_high = model_high.predict(feature_vector_df)[0]
        predicted_low = model_low.predict(feature_vector_df)[0]
        predicted_close = model_close.predict(feature_vector_df)[0]
        predicted_average = model_avg.predict(feature_vector_df)[0]
        prediction_results.insert(0,
            {
                "date": (base_date + pd.Timedelta(days=7)).strftime("%Y-%m-%d"),
                "open": predicted_open,
                "high": predicted_high,
                "low": predicted_low,
                "close": predicted_close,
                "average": predicted_average,
            }
        )
    return prediction_results

In [90]:
predictions = predict_next_7_days(selected_date, df)
predictions

[{'date': '2024-05-14',
  'open': 62867.960383383426,
  'high': 66439.58936101399,
  'low': 58312.09568298742,
  'close': 62821.12980241116,
  'average': 62674.10766335501},
 {'date': '2024-05-15',
  'open': 62628.97303727953,
  'high': 65923.55502310424,
  'low': 58353.68634308131,
  'close': 62278.82472110378,
  'average': 62325.712595186356},
 {'date': '2024-05-16',
  'open': 60991.23815596313,
  'high': 64623.01849153866,
  'low': 56871.85894697702,
  'close': 60889.6559549738,
  'average': 60947.75686332074},
 {'date': '2024-05-17',
  'open': 62205.03054720562,
  'high': 65881.17417832756,
  'low': 57828.29812620372,
  'close': 61933.610869389144,
  'average': 62098.93391128779},
 {'date': '2024-05-18',
  'open': 59774.52579742066,
  'high': 63298.05373646726,
  'low': 56049.0898365481,
  'close': 59841.63704733318,
  'average': 59946.15953699511},
 {'date': '2024-05-19',
  'open': 59780.20908583107,
  'high': 62928.79894067981,
  'low': 55818.59023344336,
  'close': 59588.0266240

In [91]:
def swing_trading_strategy(data, investment_amount):
    if data.empty:
        raise ValueError("No data available")

    # Initialize trading state
    start_date = data["date"][0]
    initial_open_price = data["open"][0]
    bitcoins = investment_amount / initial_open_price
    cash = 0
    sell_executed = False
    load_executed = False
    sell_day = None
    load_day = None
    total_avg_closing_price = 0
    number_of_days = len(data.index)

    for i in range(number_of_days):
        base_date = start_date + pd.Timedelta(days=i)
        price = data[data["date"] == base_date]
        price_high = price["high"].values[0]
        price_low = price["low"].values[0]
        price_avg = price["average"].values[0]

        # Accumulate average closing prices
        total_avg_closing_price += price_avg

        # Decide when to sell: if predicted high price is 5% higher than the open price
        if not sell_executed and price_high > initial_open_price * 1.05:
            sell_executed = True
            sell_day = base_date
            cash = bitcoins * price_high
            bitcoins = 0

        # Decide when to load: after selling, if the predicted low price is 5% lower
        if sell_executed and not load_executed and price_low < price_high * 0.95:
            # Ensure not loading on the sell day
            if base_date != sell_day:
                load_executed = True
                load_day = base_date
                bitcoins = cash / price_low
                cash = 0

    # Calculate average closing price over the prediction period
    average_closing_price = total_avg_closing_price / number_of_days

    # Final valuation at the end of the prediction period or the last available day
    final_day = start_date + pd.Timedelta(days=number_of_days - 1)
    final_close_price = data.loc[data["date"] == final_day, "close"].values[0]
    final_cash = cash if cash > 0 else bitcoins * final_close_price

    # Calculate profit/loss
    profit_loss = final_cash - investment_amount

    return {
        "profit_loss": profit_loss,
        "total_investment_amount": investment_amount,
        "total_exit_amount": final_cash,
        "average_closing_price": average_closing_price,
        "sell_day": str(sell_day.date()) if sell_day else "NA",
        "load_day": str(load_day.date()) if load_day else "NA",
        "final_bitcoins": bitcoins if cash == 0 else 0,
    }

In [92]:
# Preprocess for swing strategy
df = pd.DataFrame(predictions)
df["date"] = pd.to_datetime(df["date"])
df.dropna(inplace=True)
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df

,date,open,high,low,close,average
0,2024-05-14,62867.960383,66439.589361,58312.095683,62821.129802,62674.107663
1,2024-05-15,62628.973037,65923.555023,58353.686343,62278.824721,62325.712595
2,2024-05-16,60991.238156,64623.018492,56871.858947,60889.655955,60947.756863
3,2024-05-17,62205.030547,65881.174178,57828.298126,61933.610869,62098.933911
4,2024-05-18,59774.525797,63298.053736,56049.089837,59841.637047,59946.159537
5,2024-05-19,59780.209086,62928.798941,55818.590233,59588.026624,59568.872528
6,2024-05-20,60398.000997,63930.581557,56333.214650,60372.471778,60422.429195


In [93]:
initial_investment = 100000
result = swing_trading_strategy(df, initial_investment)
result

{'profit_loss': 9337.270824432897,
 'total_investment_amount': 100000,
 'total_exit_amount': 109337.2708244329,
 'average_closing_price': 61140.56747033866,
 'sell_day': '2024-05-14',
 'load_day': '2024-05-15',
 'final_bitcoins': 1.8110451270895136}